In [ ]:
from PIL import Image
from keras.models import load_model
from keras_facenet import FaceNet
import numpy as np
from numpy import asarray
from numpy import expand_dims
import pickle
import cv2

In [ ]:
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = FaceNet()

In [ ]:
myfile = open("dataset.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [ ]:
cap = cv2.VideoCapture(0)
while(1):
    _, gambar1 = cap.read()
    face = HaarCascade.detectMultiScale(gambar1,1.1,4)
    
    if len(face)>0:
        x1,y1,w,h = face[0]
    else:
        x1,y1,w,h = 1,1,10,10
        
    x1,y1 = abs(x1), abs(y1)
    x2,y2 = x1 + w, y1 + h
    
    gambar = cv2.cvtColor(gambar1, cv2.COLOR_BGR2RGB)
    gambar = Image.fromarray(gambar)
    gambar_array = asarray(gambar)
    
    wajah = gambar_array[y1:y2, x1:x2]
    
    wajah = Image.fromarray(wajah)
    wajah = wajah.resize((160,160))
    wajah = asarray(wajah)
    
    wajah = expand_dims(wajah, axis=0)
    signature = MyFaceNet.embeddings(wajah)
    
    min_dist = 100
    identity = ' '
    for key, value in database.items():
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gambar1, identity, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2, cv2.LINE_AA)
    cv2.rectangle(gambar1, (x1,y1), (x2,y2), (0,255,0), 2)
    
    cv2.imshow('res', gambar1)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()